In [1]:
import torch
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn

import pytorch_lightning as pl
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchmetrics as TM
pl.utilities.seed.seed_everything(seed=42)
import numpy as np
import pandas as pd

import sys, os
source_path = os.path.join(os.getcwd(), os.pardir, 'src')
sys.path.append(source_path)
source_path = os.path.join(os.getcwd(), os.pardir, 'preprocessing')
sys.path.append(source_path)

from models import NeuralNetwork, Trainer
from preprocess import (
    show_df, 
    date_features, 
    preprocess, 
    ToTorch, 
    get_loader, 
    ts_split,
    cont_cat_split
)

Global seed set to 42
Global seed set to 42


##  Get Data and train a Neural Network

In [2]:
ROOT_PATH = 'c:/Users/gilbe/Documents/TokyoData'


'/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/financials.csv'
'/train_files/trades.csv'

train_df = pd.read_csv(f'{ROOT_PATH}/train_files/stock_prices.csv')
train_df['Date'] = pd.to_datetime(train_df['Date']) 
train_df.set_index('Date', inplace=True)
# train_df = date_features(train_df)

train_options = pd.read_csv(f'{ROOT_PATH}/train_files/options.csv', low_memory=False)
train_financials = pd.read_csv(f'{ROOT_PATH}/train_files/financials.csv', low_memory=False)
train_trades = pd.read_csv(f'{ROOT_PATH}/train_files/trades.csv', low_memory=False)


import matplotlib.pyplot as plt

df_1301 = train_df[train_df['SecuritiesCode'] == 1301].drop(['SecuritiesCode', 'Volume'], axis=1)

df_1301 = date_features(df_1301)

cont, cat = cont_cat_split(df_1301, 'int64')
df_train_cat, df_val_cat = ts_split(cat)
df_train, df_val = ts_split(cont)


xtrain, ytrain = preprocess(df_train, 'Target', 1, continous_cols=['Close'])
xval, yval = preprocess(df_val, 'Target', 1, continous_cols=['Close'])


print('xtrain.shape:', xtrain.shape)
print(xtrain[:5])
print()
print('ytrain.shape:', ytrain.shape)
print('df_train_cat.shape:', df_train_cat.shape)
print(df_train_cat.head())

""" xtrain and df_train_cat have different shapes!!!!!"""

no_rows: 901
no_rows: 301
xtrain.shape: (900, 1)
[[-0.00145879]
 [ 0.00073046]
 [ 0.00291971]
 [-0.0010917 ]
 [-0.00510018]]

ytrain.shape: (901, 1)
df_train_cat.shape: (901, 3)
            day_of_year  month  day_of_week
Date                                       
2017-01-04            4      1            4
2017-01-05            5      1            5
2017-01-06            6      1            6
2017-01-10           10      1           10
2017-01-11           11      1           11


' xtrain and df_train_cat have different shapes!!!!!'

### Train the model

In [3]:
import torch
from sklearn.impute import SimpleImputer


imp = SimpleImputer(missing_values=np.nan, strategy='mean')
batch_size = 8
train_dataloader = get_loader(x=xtrain, y=ytrain, batch_size=batch_size, x_cat=df_train_cat.to_numpy())
val_dataloader = get_loader(x=xval, y=yval, batch_size=batch_size, x_cat=df_val_cat.to_numpy())


embedding_dim = 2
cat_features = 3 
cat_features = cat_features * embedding_dim
print('in_features:', xtrain.shape[1] + cat_features)

model = NeuralNetwork(
    in_features=xtrain.shape[1], 
    units=5 ,
    out_features=1, 
    num_embedding=3, 
    embedding_dim=embedding_dim
)

print(model)

trainer = Trainer(model, lr=3e-4)
trainer.fit_one_epoch(train_dataloader, val_dataloader, use_cyclic_lr=True, x_cat=True)


in_features: 7
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (embedding): Embedding(3, 2)
  (emb_input): Linear(in_features=6, out_features=8, bias=True)
  (emb_output): Linear(in_features=5, out_features=1, bias=True)
  (cont_input): Linear(in_features=1, out_features=5, bias=True)
  (hidden_layer): Linear(in_features=7, out_features=7, bias=True)
  (output_layer): Linear(in_features=7, out_features=1, bias=True)
)
Using cuda-device
x.shape: torch.Size([8, 1])
x_cat.shape before flatten: torch.Size([8, 3])
x_cat after embedding: torch.Size([8, 6])


RuntimeError: CUDA error: CUBLAS_STATUS_NOT_INITIALIZED when calling `cublasCreate(handle)`